In [2]:
import numpy as np 
import pandas as pd 
from statsmodels.stats.anova import AnovaRM
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [13]:
data1 = pd.read_csv('Utvärdering av AI-sammanfattningar (1).csv')
data2 = pd.read_csv('Utvärdering av AI-sammanfattningar (2).csv')
data3 = pd.read_csv('Utvärdering av AI-sammanfattningar (3).csv')
data4 = pd.read_csv('Utvärdering av AI-sammanfattningar (4).csv')
data = pd.concat([data1,data2,data3,data4],ignore_index=True)
df = data.filter(regex=("Summering"))
print(df)

  Hur bra tycker du summeringen är överlag? [Summering A]  \
0                                                  7        
1                                   1: Inte alls bra        
2                                                  4        
3                                                  8        
4                                                  3        
5                                                NaN        
6                                                NaN        

   Hur bra tycker du summeringen är överlag? [Summering B]  \
0                                                3.0         
1                                                3.0         
2                                                3.0         
3                                                5.0         
4                                                5.0         
5                                                NaN         
6                                                NaN         

  Hur bra tyck

In [10]:
print(data['Hur bra tycker du summeringen är överlag? [Summering A]'])

0                   7
1    1: Inte alls bra
2                   4
3                   8
4                   3
5                 NaN
6                 NaN
Name: Hur bra tycker du summeringen är överlag? [Summering A], dtype: object


In [11]:
config = [[1,2,0],[2,1,0],[0,2,1],[0,2,1],[1,2,0],[1,2,0],[0,2,1],[0,1,2],[1,0,2],[0,2,1]]

In [14]:
to_drop = ["Vad är din övergripande arbetstitel? (t.ex sjuksjöterska, läkare)",
"Vad är din ålder?",
"Hur många år har du arbetat inom sjukvården?",
"Ifall du är intresserad finns det fem block till att utvärdera. Vill du fortsätta till nästa eller lämna in?",
"Timestamp",
"Username",
]

data.drop(to_drop,inplace=True,axis=1)
data

,Hur bra tycker du summeringen är överlag? [Summering A],Hur bra tycker du summeringen är överlag? [Summering B],Hur bra tycker du summeringen är överlag? [Summering C],Hur relevant tyckte du innehållet i summeringen var? [Summering A],Hur relevant tyckte du innehållet i summeringen var? [Summering B],Hur relevant tyckte du innehållet i summeringen var? [Summering C],"Innehåller summeringen någon falsk information? Alltså, information som ej finns i anteckningarna återges i summeringen [Summering A]","Innehåller summeringen någon falsk information? Alltså, information som ej finns i anteckningarna återges i summeringen [Summering B]","Innehåller summeringen någon falsk information? Alltså, information som ej finns i anteckningarna återges i summeringen [Summering C]",Saknar sammanfattningen något som borde varit med? [Summering A],...,Saknar sammanfattningen information som borde varit med? [Summering B].7,Saknar sammanfattningen information som borde varit med? [Summering C].7,Hade sammanfattningen varit användbar i ditt arbete? [Summering A].9,Hade sammanfattningen varit användbar i ditt arbete? [Summering B].9,Hade sammanfattningen varit användbar i ditt arbete? [Summering C].9,Hur väl förhåller sig summeringen till medicinsk terminologi? [Summering A].9,Hur väl förhåller sig summeringen till medicinsk terminologi? [Summering B].9,Hur väl förhåller sig summeringen till medicinsk terminologi? [Summering C].9,Tidstämpel,Användarnamn
0,7,3.0,1: Inte alls bra,4.0,3.0,2.0,1: Inte alls,2,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1: Inte alls bra,3.0,2,2.0,3.0,2.0,1: Inte alls,1: Inte alls,2,5: Ingenting saknas,...,2.0,3.0,1: Inte alls,2.0,3.0,2.0,3.0,3.0,NaN,NaN
2,4,3.0,2,3.0,3.0,2.0,1: Inte alls,1: Inte alls,2,5: Ingenting saknas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/04/24 12:48:26 em EET,patrik.segerfelt@vgregion.se
3,8,5.0,3,4.0,4.0,4.0,1: Inte alls,1: Inte alls,1: Inte alls,5: Ingenting saknas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/05/06 11:10:14 fm EEST,maria.c.hermansson@vgregion.se
4,3,5.0,4,2.0,3.0,2.0,1: Inte alls,1: Inte alls,1: Inte alls,5: Ingenting saknas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/05/06 9:29:29 em EEST,orjan.falk@vgregion.se
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,5: Ingenting saknas,1: Inte alls,1: Inte alls,3.0,1: Inte alls,1: Inte alls,5: Helt och hållet,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1: Mycket information saknas,5: Ingenting saknas,1: Inte alls,1: Inte alls,4.0,2,2,5: Helt och hållet,2024/04/24 7:12:14 fm EET,axel.friis.liby@gmail.com


In [15]:
summa = list()
sweditron = list()
bertil = list()
complete = [sweditron,bertil,summa]

n_blocks = 10
n_summaries = 3
n_questions = 6


for index, row in data.iterrows():
    for i in range(n_blocks): # add index here
        summ = [index,i] + [None] * n_questions
        swed = [index,i] + [None] * n_questions
        bert = [index,i] + [None] * n_questions
        comp = [swed,bert,summ]
        for k in range(n_questions):
            for j in range(n_summaries):
                ind = config[i][j]
                comp[ind][k+2] = row.iloc[i*n_summaries*n_questions+k*n_summaries+j]
        summa.append(summ.copy())
        sweditron.append(swed.copy())
        bertil.append(bert.copy())

In [16]:
cs = ["Id","Block","Överlag","Relevans","Falsk","Missad","Användbarhet","Konventioner"]
summa_df = pd.DataFrame(summa,columns=cs).dropna().applymap(lambda x: int(str(x)[0]))
sweditron_df = pd.DataFrame(sweditron,columns=cs).dropna().applymap(lambda x: int(str(x)[0]))
bertil_df = pd.DataFrame(bertil,columns=cs).dropna().applymap(lambda x: int(str(x)[0]))

In [17]:
print(summa_df)

    Id  Block  Överlag  Relevans  Falsk  Missad  Användbarhet  Konventioner
0    0      0        3         3      2       1             2             4
1    0      1        2         2      1       1             1             3
5    0      5        1         2      1       5             1             4
6    0      6        3         2      1       1             1             4
7    0      7        1         2      1       1             1             3
10   1      0        3         3      1       3             3             5
11   1      1        1         1      1       3             1             5
12   1      2        1         1      2       1             1             3
13   1      3        2         2      1       1             1             5
14   1      4        1         1      1       1             1             1
15   1      5        1         1      1       2             1             4
16   1      6        1         2      1       2             2             5
17   1      

In [18]:
#summa_df['model'] = ["summa"]*len(summa_df)
#sweditron_df['model'] = ["sweditron"]*len(sweditron_df)
#bertil_df['model'] = ["bertil"]*len(bertil_df)
complete_df = summa_df.merge(sweditron_df,on=["Id","Block"],suffixes=('','_sweditron'))
complete_df = complete_df.merge(bertil_df,on=["Id","Block"],suffixes=('_summa','_bertil'))

In [19]:
print(complete_df.iloc[0])

Id                        0
Block                     0
Överlag_summa             3
Relevans_summa            3
Falsk_summa               2
Missad_summa              1
Användbarhet_summa        2
Konventioner_summa        4
Överlag_sweditron         1
Relevans_sweditron        2
Falsk_sweditron           2
Missad_sweditron          1
Användbarhet_sweditron    2
Konventioner_sweditron    3
Överlag_bertil            7
Relevans_bertil           4
Falsk_bertil              1
Missad_bertil             4
Användbarhet_bertil       4
Konventioner_bertil       5
Name: 0, dtype: int64


In [20]:
complete_df.to_csv("complete.csv")